In [1]:
kdprov = '76'

In [2]:
# Parameters
kdprov = "12"


In [3]:
import pandas as pd

import geopandas as gpd
from glob import glob

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [4]:
# open data sp
data_peta_w_ruta = gpd.read_file("data/output/0_raster_data_desa/desa_pend_rasterize_"+kdprov+".gpkg").to_crs('ESRI:54009')#.area
data_peta_w_ruta['idkab']=data_peta_w_ruta.iddesa.apply(lambda y: str(y)[:4])


In [5]:
data_peta_w_ruta.head(4)

,iddesa,DN,id,pop2020,geometry,idkab
0,1201060015,0.0,0.0,2292.0,"POLYGON ((9801900.000 124200.000, 9802200.000 ...",1201
1,1201060016,1.0,1.0,387.0,"POLYGON ((9799400.000 123800.000, 9800100.000 ...",1201
2,1201060017,2.0,2.0,474.0,"MULTIPOLYGON (((9796300.000 125100.000, 979630...",1201
3,1201060018,3.0,3.0,761.0,"POLYGON ((9794700.000 124200.000, 9794900.000 ...",1201


In [6]:
data_builtup_kabkota=glob("data/input/GHSL Data/2020/gpkg/ghs_built_"+kdprov+"*_nonzero.shp")

In [7]:
data_builtup_kabkota

['data/input/GHSL Data/2020/gpkg/ghs_built_1215_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_1209_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_1220_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_1216_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_1205_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_1203_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_1288_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_1202_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_1210_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_1206_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_1212_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_1225_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_1222_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_1278_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_1207_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_1274_nonzero.shp',
 'data/i

In [8]:
import os.path
def pop_dens(dt):
    global data_peta_w_ruta
    kdkab=dt.split('/')[5].replace('ghs_built_','').replace('_nonzero.shp','')
    if ((os.path.exists("data/output/1_population_density/grid_pop_"+kdkab+"_100m.gpkg")==False) 
        or (os.path.exists("data/output/1_population_density/grid_pop_"+kdkab+"_100m.gpkg-journal")==True)):
        data_builtup=gpd.read_file(dt)#.to_crs(('ESRI:54009'))
        data_builtup['DN']=data_builtup.value
        data_peta_w_ruta.pop2020.sum()
        data_builtup['centroid'] = data_builtup.centroid
        data_builtup['polygon'] = data_builtup['geometry']
        data_builtup.crs='ESRI:54009'
        data_peta_w_ruta.crs='ESRI:54009'
        data_builtup = data_builtup.loc[:,['DN','centroid','polygon']].set_geometry("centroid").sjoin(data_peta_w_ruta)
        data_builtup['TOTAL_PEND']=data_builtup['pop2020']
    # data_builtup = data_builtup.loc[:,['DN','geometry']].sjoin(data_peta_w_ruta)
        data_builtup['p_area'] = data_builtup['DN_left'] / data_builtup.groupby('iddesa').DN_left.transform('sum')
        data_builtup['j_pend'] = list(data_builtup['p_area']*data_builtup['TOTAL_PEND'])
    # data_builtup[['DN_left', 'geometry', 'iddesa','TOTAL_PEND', 'p_area','j_pend']].to_file("data/input/grid_pop_"+kdprov+"_100m.gpkg", driver="GPKG")
        data_builtup[['DN_left', 'centroid', 'iddesa','TOTAL_PEND', 'p_area','j_pend']].to_file("data/temp/grid_pop_"+kdkab+"_100m.gpkg", driver="GPKG")
        data_=data_builtup[['DN_left', 'polygon', 'iddesa','TOTAL_PEND', 'p_area','j_pend']].set_geometry("polygon")
        data_.crs='ESRI:54009'
        data_.to_file("data/output/1_population_density/grid_pop_"+kdkab+"_100m.gpkg", driver="GPKG")
    else:
        print('SKIP for: ',kdkab)

In [9]:
from tqdm.contrib.concurrent import process_map
process_map(pop_dens,data_builtup_kabkota)   

  0%|          | 0/34 [00:00<?, ?it/s]

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: ESRI:54009

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: ESRI:54009

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: ESRI:54009

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: ESRI:54009

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


...

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: ESRI:54009

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: ESRI:54009

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: ESRI:54009

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: ESRI:54009

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]